

    ~copy the initial database, DON"T MODIFY DIRECTLY.
    *Delete videos that are not available.


    *Process videos by cropping them based on the timestamps and rename in the the following format
        ~Label-videoID-1
        if there are more than 1 videos, 
        Label-videoID-2, 3, etc;


    *Crop the width and the height of the videos based of the box coordinates provided in the dataset.





In [92]:
import cv2
import numpy as np
from numpy import asarray
import pandas as pd
from PIL import Image
import os
import math

Here is the format of the videos  I am treating
{"org_text": "emotional", "clean_text": "emotional", "start_time": 0.0, "signer_id": 20, "signer": 40, "start": 0, "end": 30, "file": "SignSchool Emotional", "label": 907, "height": 360.0, "fps": 23.976, "end_time": 1.251, "url": "www.youtube.com/watch?v=C59jcSo4fEI", "text": "emotional", "box": [0.059554219245910645, 0.2810196578502655, 1.0, 0.8543376922607422], "width": 640.0},

{<given-word>, <start time> <>}

importing the data


In [93]:
test_df = pd.read_json("Datasets/MS-ASL/MS-ASL-with-new-labels/test-file.json")

changing videos to numpy array (function):

https://github.com/dxli94/WLASL/blob/master/code/I3D/datasets/nslt_dataset.py

In [94]:
# changing an image into an array
def change_img(image_name):
    img = Image.open(f"{image_name}")
 
    # asarray() class is used to convert
    # PIL images into NumPy arrays
    numpydata = asarray(img)
    
    # <class 'numpy.ndarray'>
    print(type(numpydata))
    
    #  shape
    print(numpydata.shape)


# change_img("test_frames/frame_330.jpg")

cutting the videos


splitting the videos into frames.

https://techtutorialsx.com/2021/04/29/python-opencv-splitting-video-frames/

In [95]:
# def load_rgb_frames_from_video(vid_root, vid, start, num, resize=(256, 256)):
#     video_path = os.path.join(vid_root, vid + '.mp4')

#     vidcap = cv2.VideoCapture(video_path)

#     frames = []

#     total_frames = vidcap.get(cv2.CAP_PROP_FRAME_COUNT)

#     vidcap.set(cv2.CAP_PROP_POS_FRAMES, start)
#     for offset in range(min(num, int(total_frames - start))):
#         success, img = vidcap.read()

#         w, h, c = img.shape
#         if w < 226 or h < 226:
#             d = 226. - min(w, h)
#             sc = 1 + d / min(w, h)
#             img = cv2.resize(img, dsize=(0, 0), fx=sc, fy=sc)

#         if w > 256 or h > 256:
#             img = cv2.resize(img, (math.ceil(w * (256 / w)), math.ceil(h * (256 / h))))

#         img = (img / 255.) * 2 - 1

#         frames.append(img)

#     return np.asarray(frames, dtype=np.float32)

# #num is the number of frames
# li = load_rgb_frames_from_video("IMAGE_vision", "test-video.mp4", 0, 60)
# for i in li:
#     print(i)
#     print("hi")
#     type(li)

In [96]:
def get_image(video_url, video_path, start_time, end_time, fps):
    capture = cv2.VideoCapture(f"Datasets/test_video_data/{video_url}.mp4")


    frameNr = 0
    # fps=int(capture.get(cv2.CAP_PROP_FRAME_COUNTCAP))

    #time in seconds
    start_time=start_time
    end_time=end_time

    #size of the image
    # frameWidth = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    # frameHeight = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
    # frameWidth = width
    # frameHeight = height
    
    frameCount = (end_time-start_time)

    start_frame=start_time*fps
    end_frame=end_time*fps

    # buf = np.empty((frameCount, frameHeight, frameWidth, 3), np.dtype('uint8'))

    frames=[]
    while (True):
        success, img = capture.read()
        # print(frameNr)
        if success and frameNr>start_frame and frameNr<end_frame:
            if frameNr%30==0: 
                # #transform into numpy array
                w, h, c = img.shape

                if w < 226 or h < 226:
                    d = 226. - min(w, h)
                    sc = 1 + d / min(w, h)
                    img = cv2.resize(img, dsize=(0, 0), fx=sc, fy=sc)

                if w > 256 or h > 256:
                    img = cv2.resize(img, (math.ceil(w * (256 / w)), math.ceil(h * (256 / h))))

                img = (img / 255.) * 2 - 1

                frames.append(img)

                #this line used to write it into a folder
                # cv2.imwrite(f'{video_url}/frame_{frameNr}.jpg', img)

        #end of the video, or end of the portion of the video
        if frameNr>end_frame:
            break
    
        frameNr = frameNr+1

    capture.release()


    #return a numpy array.
        #saves the video in label/url.npy
    mode=0o666

    
    try:
        #if file doesn't exist, create it   
        if os.path.isdir(f"test_data/{video_path}") == False:
            os.mkdir(f"test_data/{video_path}", mode)
        np.save(f"test_data/{video_path}/{video_url}", np.asarray(frames, dtype=np.float32), allow_pickle=True, fix_imports=True)
    
    except FileNotFoundError:
        pass

    return np.asarray(frames, dtype=np.float32)
        



#just give me the video path in here
# get_image(" ALY PUT THE VIDEO PATH, RUN SOME LOOP OR SOMETHING, and it should be fine.", "VIDEO URL (which will be the name of the folder)", "start_time", "endtime")

# get_image("alkfjdsl2", "test-video2", 5, 30, 29.97)

creating the folders for the labels


In [97]:
test_df = pd.read_json('Datasets/MS-ASL/MS-ASL-with-new-labels/NEW-MSASL_test.json')
test_df.head()

,org_text,clean_text,start_time,signer_id,signer,start,end,file,label,height,fps,end_time,url,text,box,width,review,available
0,beer,beer,0.0,20,40,0,59,SignSchool Beer var,805,360,29.970,1.969,www.youtube.com/watch?v=wX78EPtSuzU,beer,"[0.0467872322, 0.2904092073, 1, 0.8233144283]",640,NaN,False
1,enjoy,enjoy,0.0,11,39,0,54,enjoy(1),192,360,29.970,1.802,www.youtube.com/watch?v=OL02Odh2dRg,enjoy,"[0.07145091890000001, 0.1360249817, 1, 0.90551...",480,NaN,False
2,emotional,emotional,0.0,20,40,0,30,SignSchool Emotional,907,360,23.976,1.251,www.youtube.com/watch?v=C59jcSo4fEI,emotional,"[0.0595542192, 0.2810196579, 1, 0.8543376923]",640,NaN,False
3,key,key,0.0,92,-1,0,37,SignSchool Key,456,360,23.976,1.543,www.youtube.com/watch?v=Qs2ua1S6tg0,key,"[0.11462894080000001, 0.3089874387, 1, 0.82752...",640,NaN,False
4,bad,bad,0.0,11,39,0,36,BAD,81,360,29.970,1.201,www.youtube.com/watch?v=-kgTBeOw95A,bad,"[0.0404534936, 0.2204737961, 0.9983196259, 0.7...",480,NaN,False


In [98]:
hi="hello/hi"
str(hi)
hi=hi.replace("/", " ")
print(hi)
hi=hi[:-1]
print(hi)

hello hi
hello h


In [99]:
for idx in range(0, len(test_df)-1):
    #if the file is available:
    if test_df.loc[idx, "available"]==False:
        continue

    #file-name
    file_name=test_df.loc[idx, "url"][-11:]#[24:]
    # org_text=org_text.replace("/", " ")


    start_time = test_df.loc[idx, "start_time"]

    end_time=test_df.loc[idx, "end_time"]

    fps=test_df.loc[idx, "fps"]

    clean_text=test_df.loc[idx, "clean_text"]
    #replacing all odd characters
    str(org_text)
    clean_text=clean_text.replace("/", " ")
    if org_text[-1]==" ":
        org_text[:-1]

    if idx%1000==0:
        print(idx) 

    get_image(file_name, clean_text, start_time, end_time, fps)
    

    